### Шаг 0. План работ

1. **Импортировать необходимые библиотеки**
    - Импортировать пандас, numpy, os SQL-алхимию
2. **Подключиться к базе данных**
    - Определиться с типом подключения к БД
    - Создать движок SQL-алхимии
    - Проверить работоспособность на простом запросе
3. **Написать запрос по задаче**
    - Написать запрос к базе данных, который создаст таблицу-витрину для Tableau
    - Выгрузить витрину в CSV-файл
4. **Подключить CSV-файл к Tableau и создать презентацию, которая ответит на вопросы из задачи**
    1. Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
    2. Какие группы пользователей наиболее лояльны к сервису? Какие менее?
    3. Какой общий NPS среди всех опрошенных?
    4. Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
5. **Упаковать ответы на вопросы в презентацию и закрепить в JN-документе**

### Шаг 1. Импорты

In [72]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### Шаг 2. Подключение к базе данных

#### 1. Подключение

In [9]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

#### 2. Проверка

In [74]:
query = """ SELECT * FROM user LIMIT 5"""

In [76]:
df = pd.read_sql(query, engine)

display(
    df.info(),
    df
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         5 non-null      object 
 1   lt_day          5 non-null      int64  
 2   age             5 non-null      float64
 3   gender_segment  5 non-null      float64
 4   os_name         5 non-null      object 
 5   cpe_type_name   5 non-null      object 
 6   location_id     5 non-null      int64  
 7   age_gr_id       5 non-null      int64  
 8   tr_gr_id        5 non-null      int64  
 9   lt_gr_id        5 non-null      int64  
 10  nps_score       5 non-null      int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 568.0+ bytes


None

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10


#### 3. Вывод
Всё работает корректно!

### Шаг 3. Создать витрину

#### 1. Запрос

In [79]:
query = """

SELECT user_id,
       lt_day,
       CASE
           WHEN lt_day <= 365 THEN 'new'
           WHEN lt_day > 365 THEN 'old'
       END AS is_new,
       age,
       CASE
           WHEN gender_segment = 1 THEN 'woman'
           WHEN gender_segment = 0 THEN 'man'
           ELSE 'undefined'
       END as gender_segment,
       os_name,
       cpe_type_name,
       l.country,
       l.city,
       SUBSTR(ages.title, INSTR(ages.title, ' ') + 1) AS age_segment,
       SUBSTR(ts.title, INSTR(ts.title, ' ') + 1) AS traffic_segment,
       lts.lt_gr_id AS lifetime_segment,
       nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'promoter'
           WHEN nps_score > 6 THEN 'passive'
           WHEN nps_score <= 6 THEN 'detractor'
       END AS nps_group

  FROM user AS u
  JOIN location AS l 
    ON u.location_id=l.location_id
  JOIN age_segment AS ages 
    ON u.age_gr_id=ages.age_gr_id
  JOIN traffic_segment AS ts 
    ON u.tr_gr_id=ts.tr_gr_id
  JOIN lifetime_segment AS lts 
    ON u.lt_gr_id=lts.lt_gr_id
  
"""

#### 2. Вывод таблицы

In [80]:
df = pd.read_sql(query, engine)

display(
    df.info(),
    df.sample(25)
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  int64  
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 53.7+ MB


None

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
340647,RMKKQC,222,new,30.0,man,ANDROID,SMARTPHONE,Россия,Москва,25-34,40-45,5,10,promoter
237565,MAB47R,777,old,30.0,woman,ANDROID,SMARTPHONE,Россия,НижнийНовгород,25-34,35-40,7,10,promoter
352321,S8B2S0,1676,old,41.0,man,ANDROID,SMARTPHONE,Россия,Москва,35-44,5-10,8,10,promoter
291603,P2X8EO,3983,old,30.0,woman,ANDROID,SMARTPHONE,Россия,Волгоград,25-34,15-20,8,8,passive
356378,SFP7E2,4211,old,44.0,woman,ANDROID,SMARTPHONE,Россия,Хабаровск,35-44,1-5,8,5,detractor
365597,SWRKYD,4016,old,61.0,woman,ANDROID,SMARTPHONE,Россия,Балашиха,55-64,1-5,8,10,promoter
364696,SV4ODC,653,old,38.0,woman,ANDROID,SMARTPHONE,Россия,Тольятти,35-44,30-35,6,5,detractor
338674,RIX2K4,2148,old,41.0,man,ANDROID,SMARTPHONE,Россия,Архангельск,35-44,5-10,8,7,passive
168689,IPH7GP,736,old,25.0,woman,ANDROID,SMARTPHONE,Россия,Краснодар,25-34,60-65,7,10,promoter
1756,A3A042,2629,old,32.0,man,ANDROID,SMARTPHONE,Россия,Ижевск,25-34,60-65,8,5,detractor


Всё работает корректно!

#### 3. Выгрузка таблицы в CSV

In [82]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Шаг 4. Ссылка на дашборд

[Ссылка](https://public.tableau.com/app/profile/roman.khateev/viz/sborny_project/sheet24?publish=yes)